Son invariantes de rotación, lo que significa que, incluso si se gira la imagen, podemos encontrar las mismas esquinas. Es obvio porque las esquinas también siguen siendo esquinas en la imagen rotada. Pero ¿qué pasa con la escala? Es posible que una esquina no sea una esquina si se escala la imagen. Por ejemplo, consulte una imagen simple a continuación. Una esquina de una imagen pequeña dentro de una ventana pequeña es plana cuando se hace zoom en la misma ventana. Entonces la esquina de Harris no es invariante de escala.

1. Detección de extremos en el espacio de escala.

De la imagen de arriba, es obvio que no podemos usar la misma ventana para detectar puntos clave con diferentes escalas. Está bien con un rincón pequeño. Pero para detectar rincones más grandes necesitamos ventanas más grandes. Para ello, se utiliza el filtrado de espacio de escala. En él se encuentra el Laplaciano de Gauss para la imagen con variosσvalores. LoG actúa como un detector de manchas que detecta manchas de varios tamaños debido al cambio enσ. En breve,σactúa como parámetro de escala. Por ejemplo, en la imagen de arriba, el núcleo gaussiano con bajaσda un valor alto para esquinas pequeñas mientras que el núcleo gaussiano con altoσSe adapta bien a esquinas más grandes. Entonces, podemos encontrar los máximos locales en la escala y el espacio, lo que nos da una lista de( x , y, σ)valores, lo que significa que hay un punto clave potencial en (x,y) enσescala.

Pero este LoG es un poco costoso, por lo que el algoritmo SIFT utiliza la diferencia de gaussianos, que es una aproximación de LoG. La diferencia de Gauss se obtiene como la diferencia de desenfoque gaussiano de una imagen con dos imágenes diferentes.σ, déjalo serσyk σ. Este proceso se realiza para diferentes octavas de la imagen en la Pirámide Gaussiana. Está representado en la siguiente imagen:

Una vez que se encuentra este DoG, se buscan imágenes en busca de extremos locales en escala y espacio. Por ejemplo, un píxel de una imagen se compara con sus 8 vecinos, así como con 9 píxeles en la siguiente escala y 9 píxeles en escalas anteriores. Si es un extremo local, es un punto clave potencial. Básicamente significa que el punto clave está mejor representado en esa escala. 

2. Localización de puntos clave.

Una vez que se encuentran las posibles ubicaciones de los puntos clave, es necesario refinarlas para obtener resultados más precisos. Utilizaron la expansión de la serie de Taylor del espacio de escala para obtener una ubicación más precisa de los extremos, y si la intensidad en estos extremos es menor que un valor umbral (0,03 según el artículo), se rechaza. Este umbral se llama contrasteUmbral en OpenCV

DoG tiene una mayor respuesta a los bordes, por lo que también es necesario eliminar los bordes. Para ello se utiliza un concepto similar al detector de esquinas de Harris. Usaron una matriz de Hesse (H) de 2x2 para calcular la curvatura principal. Sabemos por el detector de esquinas de Harris que para los bordes, un valor propio es mayor que el otro. Entonces aquí usaron una función simple,

Si esta relación es mayor que un umbral, llamado edgeThreshold en OpenCV, ese punto clave se descarta. Se da como 10 en papel.

Por lo tanto, elimina los puntos clave de bajo contraste y los puntos clave de borde y lo que queda son puntos de fuerte interés.

3. Asignación de orientación.

Ahora se asigna una orientación a cada punto clave para lograr invariancia en la rotación de la imagen. Se toma una vecindad alrededor de la ubicación del punto clave según la escala y se calcula la magnitud y dirección del gradiente en esa región. Se crea un histograma de orientación con 36 contenedores que cubren 360 grados (está ponderado por la magnitud del gradiente y una ventana circular ponderada gaussiana conσigual a 1,5 veces la escala del punto clave). Se toma el pico más alto del histograma y cualquier pico superior al 80% también se considera para calcular la orientación. Crea puntos clave con la misma ubicación y escala, pero en diferentes direcciones. Contribuye a la estabilidad del emparejamiento.

4. Descriptor de puntos clave.

Ahora se crea el descriptor de punto clave. Se toma una vecindad de 16x16 alrededor del punto clave. Está dividido en 16 subbloques de tamaño 4x4. Para cada subbloque, se crea un histograma de orientación de 8 contenedores. Por lo tanto, hay disponibles un total de 128 valores de bin. Se representa como un vector para formar un descriptor de punto clave. Además de esto, se toman varias medidas para lograr robustez frente a cambios de iluminación, rotación, etc.

5. Coincidencia de puntos clave.

Los puntos clave entre dos imágenes se relacionan identificando a sus vecinos más cercanos. Pero en algunos casos, la segunda coincidencia más cercana puede estar muy cerca de la primera. Puede suceder debido al ruido o por otras razones. En ese caso, se toma la relación entre la distancia más cercana y la segunda distancia más cercana. Si es mayor que 0,8 se rechazan. Elimina alrededor del 90% de las coincidencias falsas y descarta solo el 5% de las coincidencias correctas, según el artículo.

TAMIZAR en OpenCV.

Ahora veamos las funcionalidades SIFT disponibles en OpenCV. Tenga en cuenta que anteriormente solo estaban disponibles en el repositorio opencv contrib , pero la patente expiró en el año 2020. Por lo tanto, ahora están incluidos en el repositorio principal. Comencemos con la detección de puntos clave y dibújelos. Primero tenemos que construir un objeto SIFT. Podemos pasarle diferentes parámetros que son opcionales y están bien explicados en los documentos.

In [1]:
import numpy as np
import cv2 as cv
img = cv.imread('resources/home.jpg')
gray= cv.cvtColor(img,cv.COLOR_BGR2GRAY)
sift = cv.SIFT_create()
kp = sift.detect(gray,None)
img=cv.drawKeypoints(gray,kp,img)
cv.imwrite('resources/sift_keypoints.jpg',img)

True

La función sift.detect() encuentra el punto clave en las imágenes. Puede pasar una máscara si desea buscar solo una parte de la imagen. Cada punto clave es una estructura especial que tiene muchos atributos como sus coordenadas (x,y), tamaño del vecindario significativo, ángulo que especifica su orientación, respuesta que especifica la fuerza de los puntos clave, etc.

OpenCV también proporciona la función cv.drawKeyPoints() que dibuja pequeños círculos en las ubicaciones de los puntos clave. Si le pasa una bandera, cv.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS , dibujará un círculo con el tamaño del punto clave e incluso mostrará su orientación. Vea el ejemplo a continuación.

In [2]:
img=cv.drawKeypoints(gray,kp,img,flags=cv.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)
cv.imwrite('resources/sift_keypoints.jpg',img)

True

Ahora, para calcular el descriptor, OpenCV proporciona dos métodos.

Como ya encontró puntos clave, puede llamar a sift.compute() que calcula los descriptores a partir de los puntos clave que hemos encontrado. Por ejemplo: kp,des = sift.compute(gris,kp)
Si no encontró puntos clave, busque directamente puntos clave y descriptores en un solo paso con la función sift.detectAndCompute() .

In [3]:
sift = cv.SIFT_create()
kp, des = sift.detectAndCompute(gray,None)